In [ ]:
!pip install diffusers transformers accelerate

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install pillow

In [ ]:
!sudo apt-get install -y fonts-nanum

In [ ]:
!sudo fc-cache -fv

In [ ]:
!rm ~/.cache/matplotlib -rf

In [ ]:
plt.rc('font', family='NanumBarunGothic')

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
# 모델 로드 (일관성을 위해 시드 고정)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")


In [ ]:
# 동화 설정
story_title = "용감한 토끼 루나의 모험"
character_style = "cute cartoon style, children's book illustration, soft colors, friendly atmosphere"

In [ ]:
# 각 페이지별 스토리와 프롬프트
story_pages = [
    {
        "page": 1,
        "text": "옛날 옛적, 숲 속 작은 마을에 루나라는 용감한 토끼가 살고 있었어요.",
        "prompt": f"a brave little rabbit named Luna in a small forest village, cozy houses in background, {character_style}"
    },
    {
        "page": 2,
        "text": "어느 날, 마을의 당근밭이 신비한 안개로 덮였어요.",
        "prompt": f"mysterious fog covering a carrot garden, rabbit Luna looking worried, village in background, {character_style}"
    },
    {
        "page": 3,
        "text": "루나는 용기를 내어 안개 속으로 들어갔어요. 그곳에서 슬픈 마법사를 만났답니다.",
        "prompt": f"brave rabbit Luna meeting a sad wizard in magical fog, mystical atmosphere, {character_style}"
    },
    {
        "page": 4,
        "text": "마법사는 자신의 마법이 잘못되어 안개가 생겼다고 말했어요. 루나는 도와주기로 했어요.",
        "prompt": f"rabbit Luna helping a kind wizard, magical sparkles around them, working together, {character_style}"
    },
    {
        "page": 5,
        "text": "루나와 마법사가 힘을 합쳐 안개를 걷어내자, 마을에 다시 평화가 찾아왔어요.",
        "prompt": f"happy ending scene, rabbit Luna and wizard celebrating, clear sunny village with beautiful gardens, {character_style}"
    }
]

In [ ]:
# 이미지 생성 및 저장
def generate_storybook_images(story_pages, seed=42):
    """동화책 이미지들을 생성하고 저장하는 함수"""

    # 결과 저장할 폴더 생성
    os.makedirs("storybook_images", exist_ok=True)

    # 일관성을 위해 시드 설정
    generator = torch.Generator(device="cuda").manual_seed(seed)

    generated_images = []

    for page_info in story_pages:
        print(f"페이지 {page_info['page']} 생성 중...")
        print(f"텍스트: {page_info['text']}")

        # 이미지 생성
        image = pipe(
            page_info['prompt'],
            generator=generator,
            num_inference_steps=30,
            guidance_scale=7.5
        ).images[0]

        # 이미지 저장
        filename = f"storybook_images/page_{page_info['page']:02d}.png"
        image.save(filename)

        generated_images.append({
            'page': page_info['page'],
            'image': image,
            'text': page_info['text'],
            'filename': filename
        })

        print(f"저장됨: {filename}\n")

    return generated_images

In [ ]:
# 이미지 생성 실행
images = generate_storybook_images(story_pages)

In [ ]:
# 생성된 이미지들을 한 번에 보기
def display_storybook(images):
    """생성된 동화책 이미지들을 예쁘게 출력"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(story_title, fontsize=20, fontweight='bold')

    # 2x3 그리드로 배치 (5개 이미지 + 1개 빈 공간)
    positions = [(0,0), (0,1), (0,2), (1,0), (1,1)]

    for i, img_info in enumerate(images):
        if i < len(positions):
            row, col = positions[i]
            axes[row, col].imshow(img_info['image'])
            axes[row, col].set_title(f"페이지 {img_info['page']}", fontsize=14, fontweight='bold')
            axes[row, col].axis('off')

            # 텍스트를 이미지 아래에 추가
            axes[row, col].text(0.5, -0.15, img_info['text'],
                              transform=axes[row, col].transAxes,
                              ha='center', va='top', fontsize=10,
                              wrap=True, bbox=dict(boxstyle="round,pad=0.3",
                              facecolor="lightblue", alpha=0.7))

    # 마지막 빈 공간 숨기기
    axes[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# 동화책 이미지들 출력
display_storybook(images)

In [ ]:
# 개별 이미지도 크게 볼 수 있게
print("개별 페이지별 상세 보기:")
for img_info in images:
    plt.figure(figsize=(10, 8))
    plt.imshow(img_info['image'])
    plt.title(f"페이지 {img_info['page']}: {img_info['text']}",
              fontsize=14, fontweight='bold', pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# print(f"\n✅ 동화책 '{story_title}' 완성!")
# print("모든 이미지가 'storybook_images' 폴더에 저장되었습니다.")